In [1]:
import nibabel as nib
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from IPython.display import display
from kim_cm import commonSegment,sliceProfil,commonProfil,indexMse,loadSlice
from sliceObject import SliceObject
from display import plotsegment,displayIntensityProfil
import numpy as np

In [2]:
img1 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/sub-0903_ses-1040_haste_no_pace_ax_12.nii.gz')
img2 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/sub-0903_ses-1040_haste_no_pace_cor_9.nii.gz')
img3 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/sub-0903_ses-1040_haste_no_pace_sag_11.nii.gz')
mask1 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/brain_masks/sub-0903_ses-1040_haste_no_pace_ax_12.nii.gz')
mask2 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/brain_masks/sub-0903_ses-1040_haste_no_pace_cor_9.nii.gz')
mask3 = nib.load('../donnee/export_chloe/sub-0903_ses-1040/brain_masks/sub-0903_ses-1040_haste_no_pace_sag_11.nii.gz')

In [3]:
class Viewer3D:
    """
    Viewer 3D is a class that alow to visualize the intersection between two orthogonal slices. You can select which image 
    you want to vizualise and which slice in each images. The intersection can be visualize with and without the mask
    The class take three diffents 3D images as parameters : it can be 'axial', 'sagital' and 'coronal' and their associated
    mask
    """
    def __init__(self,listSlice): 
        
        #attributes initialisation
        self.listSlice = listSlice
        self.data_img1 = []
        self.data_img2 = []
        self.slice_img1 = listSlice[0]
        self.slice_img2 = listSlice[1]
        self.choice = 'mask'
        self.orientation1 = 'axial'
        self.orientation2 = 'sagital'
        self.numImg1 = 0
        self.numImg2 = 0
        #self.chooseSlice12(self.numImg1,self.numImg2) 
        self.error = 'mse'
        
        
        #select the two images you are interested in
        widgets.interact(
        self.chooseImage12,orientation1 = widgets.Dropdown(
        options=['axial','sagital','coronal'],
        value='sagital',
        description='Image 1:',
        disabled=False,
        ),
        orientation2  = widgets.Dropdown(
        options=['axial','sagital','coronal'],
        value='axial',
        description='Image 2:',
        disabled=False,
        ))

        #choose the intersection with mask or without mask
        widgets.interact(self.choicePlotSeg,
        choice = widgets.Dropdown(
        options=['mask','no_mask'],
        value='mask',
        description='mask ?',
        disabled=False,
        ))
        
        
        widgets.interact(self.choice_error,
            error = widgets.Dropdown(
            options=['dice','mse'],
            value='mse',
            description='error ?',
            disabled=False,
            ))
    
        
        #display lines of intersection intersection on images
        widgets.interact(self.plotSeg,
        go=widgets.ToggleButton(
        value=False,
        description='Ok!',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='check' # (FontAwesome names without the `fa-` prefix)
        ))
        
        

    """
    Method of the class Viewer3D : choose two images of interest
    """
    def chooseImage12(self,orientation1,orientation2):
        print('Hey')
        self.orientation1 = orientation1
        self.orientation2 = orientation2
        
        for img in listSlice:
            current_orientation = img.get_orientation()
            if orientation1 == current_orientation:
                self.data_img1.append(img)
            if orientation2 == current_orientation:
                self.data_img2.append(img)
        
        nbImage1 = len(self.data_img1)
        nbImage2 = len(self.data_img2)

        ListImage1 = range(0,nbImage1)
        ListImage2 = range(0,nbImage2)


        widgets.interact(
        self.chooseSlice12,numImg1 = widgets.Dropdown(
        options=ListImage1,
        value=ListImage1[1],
        description='Slice in 1:',
        disabled=False,
        ),
                
        numImg2 = widgets.Dropdown(
        options=ListImage2,
        description='Slice in 2:',
        disabled=False,
        ))
    
    """
    Method of the class Viewer3D, choose 2 slices of interest
    """
    def chooseSlice12(self,numImg1,numImg2):
        self.numImg1 = numImg1
        self.numImg2 = numImg2
        self.slice_img1 = self.data_img1[numImg1]
        self.slice_img2 = self.data_img2[numImg2]
    
    
    """
    Method of the class choicePlotSeg, choose the segment of interest
    """
    def choicePlotSeg(self,choice):
            self.choice=choice
   
    """
    Plot the segment of intersection in two images
    """
    def plotSeg(self,go):
        
        if go==True:
            
            pointImg1,pointImg2,nbpoint,ok = commonSegment(self.slice_img1,self.slice_img2)
            if ok>0 and nbpoint != 0:
                val1,index1,nbpointSlice1=sliceProfil(self.slice_img1, pointImg1, nbpoint)
                val2,index2,nbpointSlice2=sliceProfil(self.slice_img2, pointImg2, nbpoint)
                commonVal1,commonVal2,index = commonProfil(val1, index1, val2, index2,nbpoint)
                displayIntensityProfil(commonVal1,index1,commonVal2,index2,index)
            else:
                index=0
                nbpointSlice1=0
                nbpointSlice2=0
            if self.choice=='no_mask':
                title1 = 'Intersection segment for image %s without mask, slice %d' %(self.orientation1,self.numImg1)
                title2 = 'Intersection segment for image %s without mask, slice %d'%(self.orientation2,self.numImg2)
                plotsegment(self.slice_img1,pointImg1,ok,nbpoint,title1,mask=np.nan,index=np.nan,nbpointSlice=None)
                plotsegment(self.slice_img2,pointImg2,ok,nbpoint,title2,mask=np.nan,index=np.nan,nbpointSlice=None)
            elif self.choice=='mask':
                title1 = 'Intersection segment for image %s with mask,slice %d' %(self.orientation1,self.numImg1)
                title2 = 'Intersection segment for image %s with mask,slice %d' %(self.orientation2,self.numImg2)
                plotsegment(self.slice_img1,pointImg1,ok,nbpoint,title1,mask=self.slice_img1.get_mask(),index=index,nbpointSlice=nbpointSlice1)
                plotsegment(self.slice_img2,pointImg2,ok,nbpoint,title2,mask=self.slice_img2.get_mask(),index=index,nbpointSlice=nbpointSlice2)    
            if self.error=='mse':
                indexMse1,nbpoint1 = indexMse(self.slice_img1,self.listSlice)
                print(self.slice_img1.get_orientation())
                indexMse2,nbpoint2 = indexMse(self.slice_img2,self.listSlice)
                plt.figure()
                plt.plot(indexMse1)
                plt.plot(nbpoint1)
                plt.title('MSE between slice %d in image %s and its orthogonal slice') %(self.orientation1,self.numImg1)
                plt.figure()
                plt.plot(indexMse2)
                plt.plot(nbpoint2)
                plt.title('MSE between slice %d in image %s and its orthogonal slice')%(self.orientation1,self.numImg1)
            
    def choice_error(self,error):
        self.error = error
            

In [4]:
listSlice = []
loadSlice(img1,mask1,listSlice,'axial')
loadSlice(img2,mask2,listSlice,'sagital')
loadSlice(img3,mask3,listSlice,'coronal')

Viewer3D(listSlice)

interactive(children=(Dropdown(description='Image 1:', index=1, options=('axial', 'sagital', 'coronal'), value…

interactive(children=(Dropdown(description='mask ?', options=('mask', 'no_mask'), value='mask'), Output()), _d…

interactive(children=(Dropdown(description='error ?', index=1, options=('dice', 'mse'), value='mse'), Output()…

interactive(children=(ToggleButton(value=False, description='Ok!', icon='check', tooltip='Description'), Outpu…

In [5]:
len(listSlice)

107

In [6]:
#https://github.com/mohakpatel/ImageSliceViewer3D/blob/master/ImageSliceViewer3D.ipynb 